### Comparação usando Rapidfuzz entre endereços tratados (base ouro) e brutos (base CNEFE)



In [35]:
import pandas as pd
from rapidfuzz import process, fuzz
import unidecode

# Normalização de texto
def normalize(text):
    if pd.isna(text):
        return ""
    texto = unidecode.unidecode(str(text)).strip().lower()
    return texto

# Usa um dicionário para substituir abreviações
def normalizar_abreviacoes(texto):
    abreviacoes = {
        " av ": " avenida ",  
        " avn ": " avenida ",
        " r ": " rua ",
        " pc ": " praca ",
        " al ": " alameda ",
        " tr ": " travessa ",
        " jd ": " jardim ",
        " vl ": " vila "
    }
    texto = " " + texto + " " 
    for abrev, completo in abreviacoes.items():
        texto = texto.replace(abrev, completo)
    return texto.strip()  

df_bruto = pd.read_csv("3513801_DIADEMA.csv", sep=";", dtype=str)
df_tratado = pd.read_csv("cnes_geo_padrao_ouro_diadema.csv", sep=";", dtype=str)

### Unificação dos campos dos endereços para comparação

In [36]:
# Concatenação dos campos da base bruta
df_bruto["endereco_original"] = (
    df_bruto["NOM_TIPO_SEGLOGR"].fillna("") + " " +
    df_bruto["NOM_TITULO_SEGLOGR"].fillna("") + " " +
    df_bruto["NOM_SEGLOGR"].fillna("") + " " +
    df_bruto["NUM_ENDERECO"].fillna("") + " " +
    df_bruto["DSC_LOCALIDADE"].fillna("")
).apply(normalize)

# Concatenação dos campos da base tratada
df_tratado["endereco_original"] = (
    df_tratado["NO_LOGRADO"].fillna("") + " " +
    df_tratado["NU_ENDEREC"].fillna("") + " " +
    df_tratado["NO_BAIRRO"].fillna("")
).apply(normalize)

# Usa a normalização de abreviações
df_bruto["endereco_normalizado"] = df_bruto["endereco_original"].apply(normalizar_abreviacoes)  
df_tratado["endereco_normalizado"] = df_tratado["endereco_original"].apply(normalizar_abreviacoes)  

df_tratado[["endereco_original", "endereco_normalizado"]].head(10)

,endereco_original,endereco_normalizado
0,av antonio piranga 614 centro,avenida antonio piranga 614 centro
1,avenida antonio piranga 700 centro,avenida antonio piranga 700 centro
2,av antonio piranga 614 centro,avenida antonio piranga 614 centro
3,rua dos cariris 195 piraporinha,rua dos cariris 195 piraporinha
4,rua moaciar goulart cunha caldas 111 centro,rua moaciar goulart cunha caldas 111 centro
5,praca rui barbosa 27 piraporinha,praca rui barbosa 27 piraporinha
6,av nossa senhora dos navegantes 288 eldorado,avenida nossa senhora dos navegantes 288 eldorado
7,rua capibaribe 193 jardim campanario,rua capibaribe 193 jardim campanario
8,av piraporinha 1682 piraporinha,avenida piraporinha 1682 piraporinha
9,rua jose bonifacio 1641 serraria,rua jose bonifacio 1641 serraria


### Busca do endereço mais semelhtante usando o RapidFuzz
O 'fuzz.token_set_ratio' ignora a ordem das palavras, trata os casos quando um endereço está contido no outro e quando há palavras extras em um dos lados.

Ex: "Rua das Flores 123 América" vs "Rua das Flores 123 Jardim América"

In [37]:
resultados = []
for idx_tratado, row_tratado in df_tratado.iterrows():
    endereco_t_normalizado = row_tratado["endereco_normalizado"]  
    endereco_t_original = row_tratado["endereco_original"]        

    match, score, idx_bruto = process.extractOne(
        endereco_t_normalizado,                                  
        df_bruto["endereco_normalizado"],                        
        scorer=fuzz.token_set_ratio         # Há palavras extras em um dos lados, mas a maioria está presente nos dois
    )

    endereco_b_original = df_bruto.loc[idx_bruto, "endereco_original"]

    resultados.append({
        "idx_tratado": idx_tratado,
        "endereco_tratado": endereco_t_original,                
        "idx_bruto": idx_bruto,  
        "endereco_bruto": endereco_b_original,                 
        "similaridade": score,
        "endereco_tratado_normalizado": endereco_t_normalizado,
        "endereco_bruto_normalizado": df_bruto.loc[idx_bruto, "endereco_normalizado"],  
    })

### Comparação dos CEPs

In [38]:
# Normaliza os CEPs
df_tratado['CO_CEP'] = df_tratado['CO_CEP'].astype(str).str.strip().str.zfill(8).str.replace('-', '')
df_bruto['CEP'] = df_bruto['CEP'].astype(str).str.strip().str.zfill(8).str.replace('-', '')

for resultado in resultados:
    idx_tratado = resultado["idx_tratado"]
    idx_bruto = resultado["idx_bruto"]
    
    cep_tratado = df_tratado.loc[idx_tratado, "CO_CEP"]
    cep_bruto = df_bruto.loc[idx_bruto, "CEP"]
    
    # Verifica se os CEPs são iguais
    resultado["cep_tratado"] = cep_tratado
    resultado["cep_bruto"] = cep_bruto
    resultado["cep_igual"] = (cep_tratado == cep_bruto)

### Comparação da latitude e longitude

In [79]:
import math

def normalize_coordinate(coord):
    if pd.isna(coord):
        return None
    try:
        # Converte para string e remove espaços
        coord_str = str(coord).strip()

        # Substitui vírgula por ponto decimal se necessário
        coord_str = coord_str.replace(',', '.')

        # Se houver mais de um ponto, provavelmente está com separador de milhar
        if coord_str.count('.') > 1:
            # Remove todos os pontos e tenta remontar
            coord_str = coord_str.replace('.', '')
            # Insere o ponto antes dos últimos 6 dígitos (assumindo coordenada válida com 6 casas decimais)
            coord_str = coord_str[:-6] + '.' + coord_str[-6:]

        return float(coord_str)
    except:
        return None

# Formata um valor de latitude/longitude como string
def latlong_to_string(valor, br_style=False):
    """
    br_style: bool, se True retorna formato brasileiro '23.696.257'
                se False retorna formato americano '23.696257'
    """
    if pd.isnull(valor):
        return ''
    try:
        if br_style:
            s = f"{float(valor):.6f}"
            inteiro, decimal = s.split('.')
            # agrupa casas decimais de 3 em 3 e junta com pontos
            decimal_formatado = '.'.join([decimal[i:i+3] for i in range(0, len(decimal), 3)])
            return f"{inteiro}.{decimal_formatado}"
        else:
            # formato americano, ponto decimal normal
            return f"{float(valor):.6f}"
    except:
        return str(valor)

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calcula a distância em metros entre duas coordenadas usando a fórmula de Haversine.
    """
    # Raio da Terra em metros
    R = 6371000

    # Converte graus para radianos
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Diferenças
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Fórmula de Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance

# Normaliza as coordenadas
df_tratado['LAT_normalizada'] = df_tratado['nova_lat'].apply(normalize_coordinate)
df_tratado['LONG_normalizada'] = df_tratado['NU_LONGITU'].apply(normalize_coordinate)
df_bruto['LAT_normalizada'] = df_bruto['LATITUDE'].apply(normalize_coordinate)
df_bruto['LONG_normalizada'] = df_bruto['LONGITUDE'].apply(normalize_coordinate)

# Calcula a distância para cada resultado
for resultado in resultados:
    idx_tratado = resultado["idx_tratado"]
    idx_bruto = resultado["idx_bruto"]
    
    lat_tratado = df_tratado.loc[idx_tratado, "LAT_normalizada"]
    long_tratado = df_tratado.loc[idx_tratado, "LONG_normalizada"]
    lat_bruto = df_bruto.loc[idx_bruto, "LAT_normalizada"]
    long_bruto = df_bruto.loc[idx_bruto, "LONG_normalizada"]
    
    if None in [lat_tratado, long_tratado, lat_bruto, long_bruto]:
        distancia = None
    else:
        distancia = haversine_distance(lat_tratado, long_tratado, lat_bruto, long_bruto)
    
    resultado["distancia_metros"] = distancia
    resultado["lat_tratado"] = lat_tratado
    resultado["long_tratado"] = long_tratado
    resultado["lat_bruto"] = lat_bruto
    resultado["long_bruto"] = long_bruto

# Formata as coordenadas para o formato -46,597689	-> -46.597.689 (BR) ou -46,597689 -> -46.597689 (EUA)
df_tratado['LAT_formatada'] = df_tratado['LAT_normalizada'].apply(latlong_to_string)
df_tratado['LONG_formatada'] = df_tratado['LONG_normalizada'].apply(latlong_to_string)
df_bruto['LAT_formatada'] = df_bruto['LAT_normalizada'].apply(latlong_to_string)
df_bruto['LONG_formatada'] = df_bruto['LONG_normalizada'].apply(latlong_to_string)


### Visualização dos resultados

In [77]:
df_resultados = pd.DataFrame(resultados)

# Formata a similaridade
df_resultados['similaridade'] = df_resultados['similaridade'].astype(str).str.replace('.', ',')

# Adiciona as colunas de coordenadas de ambas as bases
df_resultados['lat_tratado'] = df_resultados['idx_tratado'].apply(lambda x: df_tratado.loc[x, 'LAT_formatada'])
df_resultados['long_tratado'] = df_resultados['idx_tratado'].apply(lambda x: df_tratado.loc[x, 'LONG_formatada'])
df_resultados['lat_bruto'] = df_resultados['idx_bruto'].apply(lambda x: df_bruto.loc[x, 'LAT_formatada'])
df_resultados['long_bruto'] = df_resultados['idx_bruto'].apply(lambda x: df_bruto.loc[x, 'LONG_formatada'])

df_resultados[["idx_tratado", 
               "endereco_tratado",   
               "idx_bruto",  
               "endereco_bruto",                 
               "similaridade",
               "cep_tratado",
               "cep_bruto",
               "cep_igual",
               "distancia_metros",
               "lat_tratado",
               "long_tratado",
               "lat_bruto",
               "long_bruto"]].head(5)

,idx_tratado,endereco_tratado,idx_bruto,endereco_bruto,similaridade,cep_tratado,cep_bruto,cep_igual,distancia_metros,lat_tratado,long_tratado,lat_bruto,long_bruto
0,0,av antonio piranga 614 centro,39597,avenida antonio piranga 61 centro,"98,50746268656717",09911160,09911160,True,2651.704551,-23.686369,-46.597689,-23.686363,-46.623730
1,1,avenida antonio piranga 700 centro,50282,avenida antonio piranga 700 centro,"100,0",09911160,09911160,True,1504.537368,-23.686098,-46.603650,-23.686248,-46.618424
2,2,av antonio piranga 614 centro,39597,avenida antonio piranga 61 centro,"98,50746268656717",09911160,09911160,True,1157.159622,-23.686369,-46.612366,-23.686363,-46.623730
3,3,rua dos cariris 195 piraporinha,3541,rua dos cariris 195 piraporinha,"100,0",09951420,09951420,True,3878.757634,-23.689318,-46.623000,-23.689450,-46.584908
4,4,rua moaciar goulart cunha caldas 111 centro,70850,rua moacyr goulart cunha caldas 111 conceicao,"86,36363636363636",09911450,09911450,True,995.487391,-23.690741,-46.625979,-23.690625,-46.616203


### Exporta os resultados em um arquivo CSV

In [78]:
df_resultados.to_csv("enderecos_comparados_tratado_vs_bruto.csv", sep=";", decimal=',', index=False)